In [5]:
import numpy as np
import pandas as pd

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
test = pd.read_csv("./DATA/test.csv", encoding="ISO8859-2")

In [6]:
def remove_strip(str):
    return str.replace("\r", "").replace("\n", "").strip()

def get_stop_words(f, encoding='utf-8'):
    stop_words = []
    with open(f, "r", encoding=encoding) as f_stopwords:
        for line in f_stopwords:
            line = remove_strip(line)
            stop_words.append(line.lower())
    stop_words = set(stop_words)
    print("total counts: ", len(stop_words))

    return stop_words

In [4]:
stop_words = get_stop_words("stopwords.txt")
len(stop_words)

total counts:  233


233

In [7]:
data['comment_text_sw'] = data['comment_text'].apply(
    lambda text: " ".join([w for w in text.replace(",", "").replace(".", "").replace("!", "").split() if w.lower() not in stop_words]))

test['comment_text_sw'] = test['comment_text'].apply(
    lambda text: " ".join([w for w in text.replace(",", "").replace(".", "").replace("!", "").split() if w.lower() not in stop_words]))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
countVec = CountVectorizer(stop_words=stop_words)
vec = countVec.fit_transform(data['comment_text_sw'])

<127657x183652 sparse matrix of type '<class 'numpy.int64'>'
	with 3544732 stored elements in Compressed Sparse Row format>

In [9]:
vec.vocabulary_

AttributeError: vocabulary_ not found

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=10)

X_tfidf = tfidf.fit_transform(data['comment_text'].values)
X_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names())
X_df

,00,000,000000,0001,001,004,007,01,02,03,...,śzco,šoąç,ščalk,ţr,ťé,ůź,űľó,ždia,ˇx,ˇň
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X_topics = pca.fit_transform(X_df)
X_topics

In [1]:
y_columns = data.columns[2:]
models = []

NameError: name 'data' is not defined

In [42]:
from sklearn.model_selection import train_test_split
for y_column in y_columns:
    y = data[y_column]
    X_train, X_valid, y_train, y_valid = train_test_split(X_topics, y, test_size=0.25, stratify=y)
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import f1_score
    rfc =RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_valid_pred = rfc.predict(X_valid)
    models.append(rfc)
    print("%s f1_score: %.2f" % (y_column, f1_score(y_valid, y_valid_pred) * 100))

toxic f1_score: 34.83
severe_toxic f1_score: 10.50
obscene f1_score: 29.79
threat f1_score: 0.00
insult f1_score: 22.11
identity_hate f1_score: 3.30


In [39]:
test = pd.read_csv("./DATA/test.csv", encoding='ISO8859-2')
test_tfidf = tfidf.transform(test['comment_text'].values)
test_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names())

In [66]:
test_topics = pca.transform(test_df)
result = test['id']
for (idx, column) in enumerate(y_columns):
    y_test = models[idx].predict(test_topics)
    result = pd.concat([result, pd.DataFrame(y_test, columns=[column], index=test.index)], axis=1)

In [68]:
result.to_csv("_results.csv", index=None)